## Web API's continued: Face Recognition, Entity Extraction, etc

Now let's try to play with a few APIs that are a bit more complex than the ones that we dealt earlier.

## FacePlusPlus API: Face Recognition

Let's start with the FacePlusPlus API that allows us to recognize faces. We will call the API through Mashape, which will also allow us to learn about _headers_, which is an additional piece of information that we send to APIs, in addition to parameters. The documentation of Face++ on Mashape can be found at https://market.mashape.com/faceplusplus/faceplusplus-face-detection.

We will start by analyzing the image below, which is accessible through this URL: http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg

![Image from NY Times](http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg)

In [132]:
import requests
import json

facepp_url = "https://faceplusplus-faceplusplus.p.mashape.com/detection/detect"
img_url = "http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg"

headers = {
  "X-Mashape-Key": "zG3wec50exmshxNoF1NMHNRH37GYp1d7oW8jsnWwIMTeMmALxg",
  "Accept": "application/json"
}
parameters = {
    'attributes': 'glass,pose,gender,age,race,smiling',
    'url': img_url
}

data = requests.get(facepp_url, params=parameters, headers=headers, verify=True).json()


In [133]:
data.keys()

dict_keys(['img_width', 'face', 'img_height', 'session_id', 'img_id', 'url'])

In [134]:
data['face']

[{'attribute': {'age': {'range': 7, 'value': 27},
   'gender': {'confidence': 99.9869, 'value': 'Female'},
   'race': {'confidence': 99.9008, 'value': 'White'},
   'smiling': {'value': 94.373}},
  'face_id': 'c55b52ce2f623acca70796b56d2f4e79',
  'position': {'center': {'x': 33.25, 'y': 56.967213},
   'eye_left': {'x': 30.623667, 'y': 52.593716},
   'eye_right': {'x': 35.997333, 'y': 52.868306},
   'height': 17.759563,
   'mouth_left': {'x': 30.004833, 'y': 59.921585},
   'mouth_right': {'x': 36.188, 'y': 60.390984},
   'nose': {'x': 32.874667, 'y': 58.267486},
   'width': 10.833333},
  'tag': ''},
 {'attribute': {'age': {'range': 10, 'value': 61},
   'gender': {'confidence': 99.9889, 'value': 'Male'},
   'race': {'confidence': 99.00919999999999, 'value': 'White'},
   'smiling': {'value': 43.5574}},
  'face_id': '2470bdb8c39898a536d6fa17407933f8',
  'position': {'center': {'x': 66.333333, 'y': 57.377049},
   'eye_left': {'x': 64.0845, 'y': 52.454372},
   'eye_right': {'x': 69.210667, 'y

In [135]:
# The "face" attribute contains a list, and each element of the list is a dictionary
len(data["face"])

2

#### Exercise

* Print the gender, age, race, and smiling attributes for each face
* Do an image search and get an image URL from the Internet, preferably with multiple faces. Repeat the task above for the new image.

In [ ]:
# your code here

### Interacting with the IBM Watson Natural Language Understanding API; POST vs GET

Another useful API, especially when dealing with text, is the [IBM Watson  Natural Language Understanding API](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#introduction), which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc.

#### /analyze call

We will first start with the `GET /analyze` API call ([documentation](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#get-analyze)), which takes as input a piece of text, and returns an analysis across various dimensions.

The call below gets as input a "text" variable, and returns back the sentiment of the text.

In [140]:
import requests
import json

def getSentiment(text):
    endpoint = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"

    # You can register and get your own credentials
    # The ones below have a quota of 1000 calls per day 
    # and can run out quickly if multiple people use these
    username = "802a033d-ff91-4b02-a6c4-a40703ac1b16"
    password = "TBWFrRx6xwmc"

    parameters = {
        #'features' : 'concepts,categories,emotion,entities,keywords,metadata,relations,semantic_roles,sentiment',
        'features': 'emotion,sentiment',
        'version' : '2017-02-27',
        'text': text,
        'language' : 'en',
        # url = url_to_analyze, this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

text = '''
This class is challenging, but I love how much I am learning.
'''

data = getSentiment(text)

In [141]:
data.keys()

dict_keys(['language', 'emotion', 'sentiment'])

In [142]:
data['sentiment']

{'document': {'label': 'positive', 'score': 0.766383}}

In [143]:
data['emotion']

{'document': {'emotion': {'anger': 0.004903,
   'disgust': 0.021981,
   'fear': 0.156431,
   'joy': 0.775761,
   'sadness': 0.057214}}}

#### Entities call

[Full Documentation of the call](https://www.ibm.com/watson/developercloud/natural-language-understanding/api/v1/#entities)

This is a an API call that extracts entities from the text, and also the sentiment and emotion for each of these entities. You will also see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Trump" and "Donald Trump" get mapped to the same Knowledge Graph entity.

In [145]:
import requests
import json

def processURL(url_to_analyze):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    watson_options = {
      "url": url_to_analyze,
      "features": {
        "entities": {
          "sentiment": True,
          "emotion": True,
          "limit": 10
        }
      }
    }
    username = "802a033d-ff91-4b02-a6c4-a40703ac1b16"
    password = "TBWFrRx6xwmc"

    resp = requests.post(endpoint_watson, data=json.dumps(watson_options), 
                         headers=headers, params=params, auth=(username, password) )
    return resp.json()


url_to_analyze = 'http://www.politico.com/story/2017/05/23/infrastructure-transportation-trump-budget-238741'

data = processURL(url_to_analyze)

In [146]:
# Let's see what we get back as top-level attributes
data.keys()

dict_keys(['language', 'entities', 'retrieved_url'])

In [147]:
# Let' see the entities list
data["entities"]

[{'count': 4,
  'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Donald_Trump',
   'name': 'Donald Trump',
   'subtype': ['AwardNominee',
    'AwardWinner',
    'Celebrity',
    'CompanyFounder',
    'TVPersonality',
    'TVProducer',
    'FilmActor',
    'TVActor']},
  'emotion': {'anger': 0.202945,
   'disgust': 0.290059,
   'fear': 0.284567,
   'joy': 0.050412,
   'sadness': 0.148262},
  'relevance': 0.882851,
  'sentiment': {'score': 0.0},
  'text': 'President Donald Trump',
  'type': 'Person'},
 {'count': 6,
  'emotion': {'anger': 0.105083,
   'disgust': 0.083277,
   'fear': 0.281151,
   'joy': 0.211854,
   'sadness': 0.431002},
  'relevance': 0.684575,
  'sentiment': {'score': 0.0797512},
  'text': 'president',
  'type': 'JobTitle'},
 {'count': 3,
  'emotion': {'anger': 0.0,
   'disgust': 0.0,
   'fear': 0.0,
   'joy': 0.0,
   'sadness': 0.0},
  'relevance': 0.612887,
  'sentiment': {'score': 0.51212},
  'text': 'Trump',
  'type': 'Company'},
 {'count': 2,
  'e

In [148]:
# Let' see the first entity. Notice the "disambiguated" attribute that
# points to "canonical" versions of the entity, in DBPedia, Freebase, OpenCYC, YAGO, etc
data["entities"][0]

{'count': 4,
 'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Donald_Trump',
  'name': 'Donald Trump',
  'subtype': ['AwardNominee',
   'AwardWinner',
   'Celebrity',
   'CompanyFounder',
   'TVPersonality',
   'TVProducer',
   'FilmActor',
   'TVActor']},
 'emotion': {'anger': 0.202945,
  'disgust': 0.290059,
  'fear': 0.284567,
  'joy': 0.050412,
  'sadness': 0.148262},
 'relevance': 0.882851,
 'sentiment': {'score': 0.0},
 'text': 'President Donald Trump',
 'type': 'Person'}

In [149]:
# This function takes as input the result
# from the IBM Watson API and returns a list
# of entities that are relevant (above threshold)
# to the article
def getEntities(data, threshold):
    result = []
    for entity in data["entities"]:
        relevance = float(entity['relevance'])
        if relevance > threshold:
            result.append(entity['text'])
    return result

getEntities(data, 0.25)

['President Donald Trump',
 'president',
 'Trump',
 'Trump administration',
 'Transportation Secretary Elaine Chao',
 'Congress',
 'Chuck Schumer',
 'Rep. Peter DeFazio',
 'Federal Aviation Administration']

#### Exercise

* Fetch the main page of NY Times. Print the entities that are currently being discussed in the news, together with their relevance value and the associated sentiment.
* _Optional:_ Use the NY Times API to fetch the Top Stories News. You can register and get an API key at https://developer.nytimes.com/. The `Top Stories V2 API` provides the details of the news of the day: (The API call documentation is at https://developer.nytimes.com/top_stories_v2.json and the API Call is  https://api.nytimes.com/svc/topstories/v2/home.json?api-key=PUTYOURKEYHERE). Repeat the entity extraction process from above.

### Exercise: Using the Spotify API

We will now use the Spotify API to get information about an artist. The documentation of the calls is at https://developer.spotify.com/web-api/endpoint-reference/. For now, use only the calls that do not require an OAuth authentication. 

Tasks:
* We can first find the id of an artist using the `/v1/search?type=artist` API call. The documentation of the `search-item` endpoint is at https://developer.spotify.com/web-api/search-item/.
* Once you get back the ID of the artist, use the `get artist` endpoint, to get further information about the artist: https://developer.spotify.com/web-api/get-artist/
* Study the documentation and figure out how to get the albums of an article, the top tracks for an artist, and the related artists.



